# **CHAPTER 10 <br> Design Patterns with First-Class Functions**

- In software engineering, a design pattern is a **general recipe for solving a common
design problem**. You don’t need to know design patterns to follow this chapter.

- Although design patterns are language independent, that does not mean every pattern
applies to every language. Therefore implementation language determines which patterns are relevant.
- In particular, in the context of languages with first-class functions, Norvig (“Design Patterns in Dynamic Languages”, Peter Norvig, 1996 presentation) suggests rethinking the classic patterns known as **Strategy**, **Command**, **Template Method**, and **Visitor**.

- The **goal of this chapter** is to show how—in some cases—functions can do the same work as classes, with code that is more readable and concise.

# Strategy pattern



Define a family of algorithms, encapsulate each one, and make them interchangeable.
Strategy lets the algorithm vary independently from clients that use it.

- *Context*: Provides a service by delegating some computation to interchangeable components
that implement alternative algorithms. In the ecommerce example, the
context is an Order, which is configured to apply a promotional discount according
to one of several algorithms.

- *Strategy*:The interface common to the components that implement the different algorithms.
In our example, this role is played by an abstract class called Promotion.

- *Concrete strategy*: One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo, and LargeOrderPromo are the three concrete strategies implemented.


In [1]:
# Example 10.1 Implementation of the Order class with pluggable discount strategies
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


class Promotion(ABC):  # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [2]:
# Two customers: joe has 0 fidelity points, ann has 1,100.
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = (LineItem('banana', 4, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5)))

In [3]:
# The FidelityPromo promotion gives no discount to joe.
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [4]:
# ann gets a 5% discount because she has at least 1,000 points.
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

# Function-Oriented Strategy

- Since each concrete strategy is a class with **no attribute** and **single method**, they look a lot like plain functions.

- Replacing the concrete strategies with simple functions 
and removing the Promo abstract  class is a way to refactor Example 10-1. Only small adjustments are needed in the Order class.

In [7]:
#Example 10.3 Order class with discount strategies implemented as functions

from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity

@dataclass(frozen=True)
class Order:  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None  # <1> promotion may be None, or it may be a callable

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)  # <2> promotion is not a method. It’s an instance
        return self.total() - discount

    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


def fidelity_promo(order: Order) -> Decimal:  # <4> Each strategy is a function.
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [8]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5))]
banana_cart = [LineItem('banana', 30, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5'))]

In [9]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [10]:

Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

Same test fixtures as Example 10-1.
To apply a discount strategy to an Order, just pass the promotion function as an
argument.
A different promotion function is used here and in the next test.

$\color{orange}{Note:}$  <br>

- Often concrete strategies have no internal state; they only deal with data from the context. If that is the case, then by all means use plain old functions instead of coding single-method classes implementing a single-method interface declared in yet another class. A function is more lightweight than an instance of a user-defined class, and there is no need for Flyweight because each strategy function is created just once per Python process when it loads the module. A plain function is also “a shared
object that can be used in multiple contexts simultaneously.

# Choosing the Best Strategy: Simple Approach

Suppose you want to create a “metastrategy” that selects the best available discount for a given Order.

In the following examples (6-7-8-9) we study additional refactorings that implement this requirement using a variety of approaches that leverage functions and modules as objects.

In [8]:
# Example 10-6. best_promo finds the maximum discount iterating over a list of functions
from decimal import Decimal

promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

In [9]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5))]
banana_cart = [LineItem('banana', 30, Decimal('.5')),
            LineItem('apple', 10, Decimal('1.5'))]
long_cart = [LineItem(str(item_code), 1, Decimal(1))
            for item_code in range(10)]

In [10]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [11]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [12]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

$\color{orange}{Note:}$  <br>
our *main issue with Example 10-6* is the repetition of the function names
in their definitions and then in the promos list used by the best_promo function to
determine the highest discount applicable. The repetition is problematic because
someone may add a new promotional strategy function and forget to manually add it
to the promos list—in which case, best_promo will silently ignore the new strategy,
introducing a **subtle bug** in the system (Example 10-9 solves this problem).<br>


Read on for a couple of solutions to this issue: 
- Finding Strategies in a Module
- Decorator-Enhanced Strategy Pattern

# Finding Strategies in a Module

## globals()

In [1]:
globals().items()

dict_items([('__name__', '__main__'), ('__doc__', 'Automatically created module for IPython interactive environment'), ('__package__', None), ('__loader__', None), ('__spec__', None), ('__builtin__', <module 'builtins' (built-in)>), ('__builtins__', <module 'builtins' (built-in)>), ('_ih', ['', 'globals().items()']), ('_oh', {}), ('_dh', [WindowsPath('c:/Users/amirh/Desktop/Pytorch RL Udemy course/Fluent Python Class codes/Ch 10'), WindowsPath('c:/Users/amirh/Desktop/Pytorch RL Udemy course/Fluent Python Class codes/Ch 10')]), ('In', ['', 'globals().items()']), ('Out', {}), ('get_ipython', <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x00000225C3FF2A70>>), ('exit', <IPython.core.autocall.ZMQExitAutocall object at 0x00000225C3FF25C0>), ('quit', <IPython.core.autocall.ZMQExitAutocall object at 0x00000225C3FF25C0>), ('open', <function open at 0x00000225C2744CA0>), ('_', ''), ('__', ''), ('___', ''), ('__vsc_ipynb_file__', 'c:\\Users\\amir

In [11]:
# Example 10-7. The promos list is built by introspection of the module global namespace
from decimal import Decimal

promos = [promo for name, promo in globals().items()
                if name.endswith('_promo') and
                   name != 'best_promo'
]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

In [13]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5))]
banana_cart = [LineItem('banana', 30, Decimal('.5')),
            LineItem('apple', 10, Decimal('1.5'))]
long_cart = [LineItem(str(item_code), 1, Decimal(1))
            for item_code in range(10)]

In [14]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30>

In [15]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [16]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

## inspect

Another way of collecting the available promotions would be to create a module and
put all the strategy functions there, except for best_promo.

The function inspect.getmembers returns the attributes of an object—in this case,
the promotions module—optionally filtered by a predicate (a boolean function). We
use inspect.isfunction to get only the functions from the module.

In [ ]:
# Example 10-8. The promos list is built by introspection of a new promotions module
from decimal import Decimal
import inspect
# promos list is built by introspection of a new promotions module
from strategy import Order
import promotions

promos = [func for _, func in inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

In [ ]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5))]
banana_cart = [LineItem('banana', 30, Decimal('.5')),
            LineItem('apple', 10, Decimal('1.5'))]
long_cart = [LineItem(str(item_code), 1, Decimal(1))
            for item_code in range(10)]

- Example 10-8 works regardless of the names given to the functions (Despite Example 10-7); all that matters is that the promotions module contains only functions that calculate discounts given
orders.

- Of course, this is an implicit assumption of the code. If someone were to create
a function with a different signature in the promotions module, then best_promo
would break while trying to apply it to an order.

- We could add more stringent tests to filter the functions, by inspecting their arguments
for instance. The point of Example 10-8 is not to offer a complete solution, but
to highlight one possible use of module introspection.

- A more explicit alternative to dynamically collecting promotional discount functions
would be to use a simple decorator (Example 10-9).

# Decorator-Enhanced Strategy Pattern

A more explicit alternative to dynamically collecting promotional discount functions
would be to use a simple.

This solution has several advantages over the others presented before:

- The promotion strategy functions don’t have to use special names—no need for
the _promo suffix.

- The @promotion decorator highlights the purpose of the decorated function, and
also makes it easy to temporarily disable a promotion: just comment out the
decorator.

- Promotional discount strategies may be defined in other modules, anywhere in
the system, as long as the @promotion decorator is applied to them.

In [17]:
# Example 10-9. The promos list is filled by the Promotion decorator

from decimal import Decimal
from typing import Callable


Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = []


def promotion(promo: Promotion) -> Promotion:
    promos.append(promo)
    return promo


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)


@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customers with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


@promotion
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [18]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, Decimal('.5')),
        LineItem('apple', 10, Decimal('1.5')),
        LineItem('watermelon', 5, Decimal(5))]

banana_cart = [LineItem('banana', 30, Decimal('.5')),
            LineItem('apple', 10, Decimal('1.5'))]
long_cart = [LineItem(str(item_code), 1, Decimal(1))
            for item_code in range(10)]

# The Command Pattern

- **Command** is another design pattern that can be simplified by the use of functions
passed as arguments.

- The **goal of Command** is to decouple an object that invokes an operation (the
invoker) from the provider object that implements it (the receiver).

- Quoting from Design Patterns book, “Commands are an object-oriented replacement for
callbacks.” The question is: do we need an object-oriented replacement for callbacks?
Sometimes yes, but not always.

In [ ]:

class MacroCommand:
    """A command that executes a list of commands"""
    def __init__(self, commands):
        self.commands = list(commands)

    def __call__(self):
        for command in self.commands:
            command()


- More advanced uses of the Command pattern—to support undo, for example—may
require more than a simple callback function. Even then, Python provides a couple of
alternatives that deserve consideration:

    - A callable instance like MacroCommand in Example 10-10 can keep whatever state
is necessary, and provide extra methods in addition to `__call__`.

    - A closure can be used to hold the internal state of a function between calls.

- At a high level, the approach here was similar to the one we applied to Strategy: replacing
with callables the instances of a participant class that implemented a singlemethod
interface. After all, every Python callable implements a single-method
interface, and that method is named `__call__`.

# Lecturers

1. Amirhossein Nourian [LinkedIn](https://www.linkedin.com/in/amirhnrn/)
2. Mehrna Faraji [LinkedIn](https://www.linkedin.com/in/mehranfaraji/)


present date : 2023-11-24

# Reviewers

1. Mahya Asgarian, review date: 2023-11-24 [LinkedIn](https://www.linkedin.com/in/mahya-asgarian-9a7b13249)